In [1]:
import torch
import os
import numpy as np
import pandas as pd
from warnings import simplefilter

import matplotlib.pyplot as plt


simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=FutureWarning)

In [2]:
MAIN_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(MAIN_DIR, "Solid_droplet", "Data")

In [3]:
from DataLoader import *
from Trainer import *
from Model import *
from RESNET import *

In [4]:
dtype = torch.float32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# Split the data into train, validation and test set
dataspliter = DataSpliter(DATA_DIR, device, train_frac=0.6, val_frac=0.2, test_frac=0.2, use_seed=True, seed_val=42)

# Load Data from DataLoader
train_set = dataspliter.train
val_set = dataspliter.val
test_set = dataspliter.test

train_dataset = CustomImageDataset(DATA_DIR, train_set)
val_dataset = CustomImageDataset(DATA_DIR, val_set)
test_dataset = CustomImageDataset(DATA_DIR, test_set)

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [8]:
# Get the model
model = CNNModel3()
trainer = Trainer(model, train_dataloader, val_dataloader, test_dataloader)

The device that will be used in training is Quadro P1000


In [9]:
trainer.fit(epochs=10, batch_size= 8)
trainer.save_model("model3_2.pt", "savefolderpytorch")



# model.load_state_dict(torch.load("./savefolderpytorch/model.pt"))

Epoch 1


Training: 2304it [24:16,  1.58it/s, loss=2.38e+3]                        
Validation: 768it [02:07,  6.05it/s, loss=2.72e+3]                       

Validation loss is 26.945367203818424
Epoch 2



Training: 2304it [24:22,  1.58it/s, loss=56.3]                           
Validation: 768it [02:06,  6.06it/s, loss=145]                        

Validation loss is 1.1740711737010214
Epoch 3


...

Validation loss is 0.06313487504505448
Epoch 9



Training: 2304it [35:04,  1.09it/s, loss=7.84]                        
Validation: 768it [02:19,  5.49it/s, loss=7.17]                       

Validation loss is 0.05586384154028363
Epoch 10



Training: 2304it [31:14,  1.23it/s, loss=5.61]                        
Validation: 768it [02:19,  5.49it/s, loss=5.01]                       


Validation loss is 0.060240909198506006


In [10]:
_, output, labels = trainer.val_epoch(test_dataloader)

Validation: 768it [02:23,  5.35it/s, loss=5.43]                       

Validation loss is 0.060980601572535105


In [11]:
dfresults = pd.DataFrame(np.append(output, labels, axis=1),columns=["sigmapred", "sigmatrue"])
dfresults["RMSE"] = np.sqrt((dfresults["sigmapred"]-dfresults["sigmatrue"])**2)
dfresults

,sigmapred,sigmatrue,RMSE
0,71.585854,69.25,2.335854
1,71.585854,70.00,1.585854
2,71.585854,67.75,3.835854
3,71.585854,72.75,1.164146
4,71.585854,71.00,0.585854
...,...,...,...
763,71.585854,73.50,1.914146
764,71.585854,69.50,2.085854
765,71.585854,68.50,3.085854
766,71.585854,71.50,0.085854


In [12]:

    
# train_data = pd.read_csv("./savefolderpytorch/train.csv")
# val_data = pd.read_csv("./savefolderpytorch/val.csv")

# train_data_epoch = train_data.groupby('epoch')['loss'].mean()
# val_data_epoch = val_data.groupby('epoch')['loss'].mean()

# plt.plot(train_data_epoch.index, train_data_epoch.values)
# plt.plot(val_data_epoch.index, val_data_epoch.values)
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.grid(True)
# plt.legend(['Train', 'Validation'])
# plt.show()